In [1]:
from scipy.stats import wilcoxon as wl 
import pandas as pd 

In [126]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 

df = pd.read_csv('./data/df.csv',sep=',')
df  = df.sort_values(by='time',ascending=True).reset_index(drop=True)

BATCHSIZE = 256
LOOKBACK = 10


date_target = ["2024-02-19",
"2024-03-15",
"2024-04-19",
"2024-05-17",
"2024-06-21",
"2024-07-19",
"2024-08-16",
"2024-09-20",
"2024-10-18",
"2024-11-15",
"2024-12-20",
"2025-01-17"]
df.head()

df['time'] = pd.to_datetime(df['time'])
df['date'] = df['time'].dt.date
df['date'] = df['date'].astype(str)
df['hour'] = df['time'].dt.hour
df['minute'] = df['time'].dt.minute 
df 
df['std'] = df['spread'].rolling(window=200).std()
df['volatility'] = df['std'].apply(lambda x: np.log(x))


In [127]:
# for i in range(1,5):
#     df[f'spread_shift_{i}'] = df.groupby(['hour','minute'])['spread'].shift(i)

df['Expiration_Date'] = df['date'].apply(lambda x: 1 if x in date_target else 0)
# df.fillna(0,inplace=True)


df.dropna(inplace=True)

df.head()


,time,IF00_close,IF01_close,spread,date,hour,minute,std,volatility,Expiration_Date
199,2024-01-24 14:19:00,3250.6,3245.8,-4.8,2024-01-24,14,19,2.284265,0.826044,0
200,2024-01-24 14:20:00,3245.8,3241.0,-4.8,2024-01-24,14,20,2.294348,0.830449,0
201,2024-01-24 14:21:00,3243.2,3239.0,-4.2,2024-01-24,14,21,2.296284,0.831292,0
202,2024-01-24 14:22:00,3240.8,3235.0,-5.8,2024-01-24,14,22,2.300440,0.833100,0
203,2024-01-24 14:23:00,3243.8,3238.0,-5.8,2024-01-24,14,23,2.316842,0.840205,0


In [128]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 58364 entries, 199 to 58562
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   time             58364 non-null  datetime64[ns]
 1   IF00_close       58364 non-null  float64       
 2   IF01_close       58364 non-null  float64       
 3   spread           58364 non-null  float64       
 4   date             58364 non-null  object        
 5   hour             58364 non-null  int32         
 6   minute           58364 non-null  int32         
 7   std              58364 non-null  float64       
 8   volatility       58364 non-null  float64       
 9   Expiration_Date  58364 non-null  int64         
dtypes: datetime64[ns](1), float64(5), int32(2), int64(1), object(1)
memory usage: 4.5+ MB


In [129]:
df['year'] = df['time'].dt.year 
df['month'] = df['time'].dt.month 

In [130]:
df['year_month'] = df[['year','month']].apply(lambda x: str(x[0])+ str('0'+ str(x[1])  if int(x[1])<=9 else str(x[1])),axis=1)

C:\Users\gaoxc\AppData\Local\Temp\ipykernel_10568\2220954218.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['year_month'] = df[['year','month']].apply(lambda x: str(x[0])+ str('0'+ str(x[1])  if int(x[1])<=9 else str(x[1])),axis=1)


In [131]:
df['year_month'].unique()

array(['202401', '202402', '202403', '202404', '202405', '202406',
       '202407', '202408', '202409', '202410', '202411', '202412',
       '202501'], dtype=object)

In [141]:
aa = df[df['date'].isin(date_target)]

In [133]:
for delay in range(-3,6):

    bb = pd.DataFrame()

    for date in date_target:
        if delay>0:
            date_list = sorted(df[df['date']<date]['date'].unique())
            date_ = date_list[-delay]
        elif delay<0:
            date_list = sorted(df[df['date']>date]['date'].unique())
            date_ = date_list[-delay]
        else:
            date_ = date 


        df_bb = df[df['date']==date_]
        bb = pd.concat([bb,df_bb],axis=0)
    bb 
    statistics = [] 
    pvalue = []

    for month in aa['year_month'].unique():
        if aa[aa['year_month']==month].shape==bb[bb['year_month']==month].shape:
            # print(month,wl(aa[aa['year_month']==month]['spread'],bb[bb['year_month']==month]['spread']))
            statistics.append(wl(aa[aa['year_month']==month]['spread'],bb[bb['year_month']==month]['spread']).statistic)
            pvalue.append(wl(aa[aa['year_month']==month]['spread'],bb[bb['year_month']==month]['spread']).pvalue)

    array_statistics = np.array(statistics)
    array_statistics[array_statistics<14000].mean()
    array_pvalue = np.array(pvalue)

    array_pvalue[array_pvalue<0.8].mean() 
    print("T"+ str(-delay), array_statistics.mean(),array_pvalue.mean() )







T3 2133.7083333333335 0.0018517788985494624
T2 2501.9166666666665 0.012789582204820847
T1 2726.4166666666665 3.6345997948012935e-05
T0 0.0 nan


c:\ProgramData\anaconda3\envs\zhuqw\Lib\site-packages\scipy\stats\_wilcoxon.py:172: RuntimeWarning: invalid value encountered in scalar divide
  z = (r_plus - mn) / se
C:\Users\gaoxc\AppData\Local\Temp\ipykernel_10568\3409371451.py:32: RuntimeWarning: Mean of empty slice.
  array_pvalue[array_pvalue<0.8].mean()
c:\ProgramData\anaconda3\envs\zhuqw\Lib\site-packages\numpy\_core\_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


T-1 7543.0 0.10653335366116728
T-2 6739.166666666667 0.04643173109747469
T-3 4257.75 0.06442527699736146
T-4 4389.416666666667 0.07485625048044434
T-5 3474.5833333333335 0.019734459144765647


In [136]:
[[540, 555],[540, 570],[540, 600],[630.0, 690.0],[780, 840], [885, 900],[870, 900],[840, 900]]

10.5*60, 11.5*60

(630.0, 690.0)

In [137]:
9.5*60,9.5*60+15,9.5*60+30,9.5*60+60

(570.0, 585.0, 600.0, 630.0)

In [138]:
for delay in [1]:

    bb = pd.DataFrame()

    for date in date_target:
        if delay>0:
            date_list = sorted(df[df['date']<date]['date'].unique())
            date_ = date_list[-delay]
        elif delay<0:
            date_list = sorted(df[df['date']>date]['date'].unique())
            date_ = date_list[-delay]
        else:
            date_ = date 


        df_bb = df[df['date']==date_]
        bb = pd.concat([bb,df_bb],axis=0)
    bb['minutes'] = bb['hour']*60 + bb['minute']
    aa['minutes'] = aa['hour']*60 + aa['minute']

    for periods in [[570, 585],[570, 600],[570, 630],[630.0, 690.0],[780, 840], [885, 900],[870, 900],[840, 900]]:
        start_min = periods[0]
        end_min = periods[1]
        statistics = [] 
        pvalue = []

        for month in aa['year_month'].unique():
            if aa[aa['year_month']==month].shape==bb[bb['year_month']==month].shape:
            # print(month,wl(aa[aa['year_month']==month]['spread'],bb[bb['year_month']==month]['spread']))
                aa_ = aa[(aa['year_month']==month) & (aa['minutes']>=start_min) & (aa['minutes']<end_min)]
                bb_ = bb[(bb['year_month']==month) & (bb['minutes']>=start_min) & (bb['minutes']<end_min)]
                # print(aa[aa['year_month']==month].head() )
            
                statistics.append(wl(aa_['spread'],bb_['spread']).statistic)
                pvalue.append(wl(aa_['spread'],bb_['spread']).pvalue)

        array_statistics = np.array(statistics)
        array_pvalue = np.array(pvalue)


        print("T"+ str(-delay),periods, array_statistics.mean(),array_pvalue.mean() )
        # print(array_statistics, array_pvalue)









C:\Users\gaoxc\AppData\Local\Temp\ipykernel_10568\526705317.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['minutes'] = aa['hour']*60 + aa['minute']


T-1 [570, 585] 11.916666666666666 0.07738285025116735
T-1 [570, 600] 48.375 0.0678305102677932
T-1 [570, 630] 135.70833333333334 0.0015332841721874496
T-1 [630.0, 690.0] 91.95833333333333 0.006183751691316281
T-1 [780, 840] 169.16666666666666 0.07508408840628332
T-1 [885, 900] 7.041666666666667 0.053825369667018876
T-1 [870, 900] 35.791666666666664 0.026833352614807546
T-1 [840, 900] 200.95833333333334 0.050977513862598835


In [139]:
for delay in [2]:

    bb = pd.DataFrame()

    for date in date_target:
        if delay>0:
            date_list = sorted(df[df['date']<date]['date'].unique())
            date_ = date_list[-delay]
        elif delay<0:
            date_list = sorted(df[df['date']>date]['date'].unique())
            date_ = date_list[-delay]
        else:
            date_ = date 


        df_bb = df[df['date']==date_]
        bb = pd.concat([bb,df_bb],axis=0)
    bb['minutes'] = bb['hour']*60 + bb['minute']
    aa['minutes'] = aa['hour']*60 + aa['minute']

    for periods in [[570, 585],[570, 600],[570, 630],[630.0, 690.0],[780, 840], [885, 900],[870, 900],[840, 900]]:
        start_min = periods[0]
        end_min = periods[1]
        statistics = [] 
        pvalue = []

        for month in aa['year_month'].unique():
            if aa[aa['year_month']==month].shape==bb[bb['year_month']==month].shape:
            # print(month,wl(aa[aa['year_month']==month]['spread'],bb[bb['year_month']==month]['spread']))
                aa_ = aa[(aa['year_month']==month) & (aa['minutes']>=start_min) & (aa['minutes']<end_min)]
                bb_ = bb[(bb['year_month']==month) & (bb['minutes']>=start_min) & (bb['minutes']<end_min)]
                # print(aa[aa['year_month']==month].head() )
            
                statistics.append(wl(aa_['spread'],bb_['spread']).statistic)
                pvalue.append(wl(aa_['spread'],bb_['spread']).pvalue)

        array_statistics = np.array(statistics)
        array_pvalue = np.array(pvalue)


        print("T"+ str(-delay),periods, array_statistics.mean(),array_pvalue.mean() )
        # print(array_statistics, array_pvalue)









C:\Users\gaoxc\AppData\Local\Temp\ipykernel_10568\1424751014.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['minutes'] = aa['hour']*60 + aa['minute']


T-2 [570, 585] 11.375 0.10153613965792313
T-2 [570, 600] 28.791666666666668 0.07083860430880021
T-2 [570, 630] 122.83333333333333 0.07605679358637706
T-2 [630.0, 690.0] 106.5 0.03829555974059772
T-2 [780, 840] 102.66666666666667 0.019128052243536035
T-2 [885, 900] 0.8333333333333334 0.0007887715186588704
T-2 [870, 900] 32.625 0.06640468048934511
T-2 [840, 900] 210.33333333333334 0.08532834600950535


In [142]:



for delay in range(-3,6):

    bb = pd.DataFrame()

    for date in date_target:
        if delay>0:
            date_list = sorted(df[df['date']<date]['date'].unique())
            date_ = date_list[-delay]
        elif delay<0:
            date_list = sorted(df[df['date']>date]['date'].unique())
            date_ = date_list[-delay]
        else:
            date_ = date 


        df_bb = df[df['date']==date_]
        bb = pd.concat([bb,df_bb],axis=0)
    bb 
    statistics = [] 
    pvalue = []

    for month in aa['year_month'].unique():
        if aa[aa['year_month']==month].shape==bb[bb['year_month']==month].shape:
            # print(month,wl(aa[aa['year_month']==month]['spread'],bb[bb['year_month']==month]['spread']))
            statistics.append(wl(aa[aa['year_month']==month]['volatility'],bb[bb['year_month']==month]['volatility']).statistic)
            pvalue.append(wl(aa[aa['year_month']==month]['volatility'],bb[bb['year_month']==month]['volatility']).pvalue)

    array_statistics = np.array(statistics)
    array_statistics[array_statistics<14000].mean()
    array_pvalue = np.array(pvalue)

    array_pvalue[array_pvalue<0.8].mean() 
    print("T"+ str(-delay), array_statistics.mean(),array_pvalue.mean() )








T3 2811.1666666666665 5.554025372959517e-05
T2 1674.6666666666667 1.1893777850413013e-06
T1 3194.5 0.008216553887084055
T0 0.0 nan


c:\ProgramData\anaconda3\envs\zhuqw\Lib\site-packages\scipy\stats\_wilcoxon.py:172: RuntimeWarning: invalid value encountered in scalar divide
  z = (r_plus - mn) / se
C:\Users\gaoxc\AppData\Local\Temp\ipykernel_10568\1129595333.py:32: RuntimeWarning: Mean of empty slice.
  array_pvalue[array_pvalue<0.8].mean()
c:\ProgramData\anaconda3\envs\zhuqw\Lib\site-packages\numpy\_core\_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


T-1 3978.75 0.0004711764585332106
T-2 3020.1666666666665 0.021953987805463062
T-3 3522.9166666666665 0.0677383101944317
T-4 3007.75 8.274402907759341e-05
T-5 2441.0833333333335 6.560075891864398e-06


In [143]:

for delay in [1]:

    bb = pd.DataFrame()

    for date in date_target:
        if delay>0:
            date_list = sorted(df[df['date']<date]['date'].unique())
            date_ = date_list[-delay]
        elif delay<0:
            date_list = sorted(df[df['date']>date]['date'].unique())
            date_ = date_list[-delay]
        else:
            date_ = date 


        df_bb = df[df['date']==date_]
        bb = pd.concat([bb,df_bb],axis=0)
    bb['minutes'] = bb['hour']*60 + bb['minute']
    aa['minutes'] = aa['hour']*60 + aa['minute']

    for periods in [[570, 585],[570, 600],[570, 630],[630.0, 690.0],[780, 840], [885, 900],[870, 900],[840, 900]]:
        start_min = periods[0]
        end_min = periods[1]
        statistics = [] 
        pvalue = []

        for month in aa['year_month'].unique():
            if aa[aa['year_month']==month].shape==bb[bb['year_month']==month].shape:
            # print(month,wl(aa[aa['year_month']==month]['spread'],bb[bb['year_month']==month]['spread']))
                aa_ = aa[(aa['year_month']==month) & (aa['minutes']>=start_min) & (aa['minutes']<end_min)]
                bb_ = bb[(bb['year_month']==month) & (bb['minutes']>=start_min) & (bb['minutes']<end_min)]
                # print(aa[aa['year_month']==month].head() )
            
                statistics.append(wl(aa_['volatility'],bb_['volatility']).statistic)
                pvalue.append(wl(aa_['volatility'],bb_['volatility']).pvalue)

        array_statistics = np.array(statistics)
        array_pvalue = np.array(pvalue)


        print("T"+ str(-delay),periods, array_statistics.mean(),array_pvalue.mean() )
        # print(array_statistics, array_pvalue)









C:\Users\gaoxc\AppData\Local\Temp\ipykernel_10568\977503683.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['minutes'] = aa['hour']*60 + aa['minute']


T-1 [570, 585] 5.166666666666667 0.0779266357421875
T-1 [570, 600] 22.333333333333332 0.04300480103120208
T-1 [570, 630] 80.83333333333333 0.006542180369328871
T-1 [630.0, 690.0] 66.25 0.013308706351767126
T-1 [780, 840] 124.33333333333333 0.0019249968242541062
T-1 [885, 900] 0.0 6.103515625e-05
T-1 [870, 900] 0.0 1.862645149230957e-09
T-1 [840, 900] 0.0 1.6295557943119348e-11


In [144]:

for delay in [2]:

    bb = pd.DataFrame()

    for date in date_target:
        if delay>0:
            date_list = sorted(df[df['date']<date]['date'].unique())
            date_ = date_list[-delay]
        elif delay<0:
            date_list = sorted(df[df['date']>date]['date'].unique())
            date_ = date_list[-delay]
        else:
            date_ = date 


        df_bb = df[df['date']==date_]
        bb = pd.concat([bb,df_bb],axis=0)
    bb['minutes'] = bb['hour']*60 + bb['minute']
    aa['minutes'] = aa['hour']*60 + aa['minute']

    for periods in [[570, 585],[570, 600],[570, 630],[630.0, 690.0],[780, 840], [885, 900],[870, 900],[840, 900]]:
        start_min = periods[0]
        end_min = periods[1]
        statistics = [] 
        pvalue = []

        for month in aa['year_month'].unique():
            if aa[aa['year_month']==month].shape==bb[bb['year_month']==month].shape:
            # print(month,wl(aa[aa['year_month']==month]['spread'],bb[bb['year_month']==month]['spread']))
                aa_ = aa[(aa['year_month']==month) & (aa['minutes']>=start_min) & (aa['minutes']<end_min)]
                bb_ = bb[(bb['year_month']==month) & (bb['minutes']>=start_min) & (bb['minutes']<end_min)]
                # print(aa[aa['year_month']==month].head() )
            
                statistics.append(wl(aa_['volatility'],bb_['volatility']).statistic)
                pvalue.append(wl(aa_['volatility'],bb_['volatility']).pvalue)

        array_statistics = np.array(statistics)
        array_pvalue = np.array(pvalue)


        print("T"+ str(-delay),periods, array_statistics.mean(),array_pvalue.mean() )
        # print(array_statistics, array_pvalue)









C:\Users\gaoxc\AppData\Local\Temp\ipykernel_10568\1034167057.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['minutes'] = aa['hour']*60 + aa['minute']


T-2 [570, 585] 1.75 0.0021870930989583335
T-2 [570, 600] 24.416666666666668 0.06472166496669235
T-2 [570, 630] 253.91666666666666 0.1002182359740119
T-2 [630.0, 690.0] 1.6666666666666667 1.8636673696078972e-11
T-2 [780, 840] 59.916666666666664 9.10710324731936e-06
T-2 [885, 900] 0.0 6.103515625e-05
T-2 [870, 900] 0.0 1.862645149230957e-09
T-2 [840, 900] 34.5 1.8823252183034162e-05


In [94]:
# for date in df_month_test['date'].unique():
#     test_spread = df_month_test[df_month_test['date']==date]['spread'].values 
#     target_spread = df_month_target['spread'].values 
#     if test_spread.shape==target_spread.shape:
#         print(date, wl(target_spread,test_spread))
#     else:
#         print(date)

In [26]:
df_month_test['hour_minte'] = df_month_test['time'].apply(lambda x: str(x)[10:])
df_month_test

,time,IF00_close,IF01_close,spread,date,hour,minute,Expiration_Date,year,month,year_month,hour_minte
14942,2024-05-06 09:30:00,3666.0,3651.4,-14.6,2024-05-06,9,30,0,2024,5,202405,09:30:00
14943,2024-05-06 09:31:00,3656.8,3646.2,-10.6,2024-05-06,9,31,0,2024,5,202405,09:31:00
14944,2024-05-06 09:32:00,3656.8,3646.8,-10.0,2024-05-06,9,32,0,2024,5,202405,09:32:00
14945,2024-05-06 09:33:00,3653.8,3643.2,-10.6,2024-05-06,9,33,0,2024,5,202405,09:33:00
14946,2024-05-06 09:34:00,3657.0,3647.0,-10.0,2024-05-06,9,34,0,2024,5,202405,09:34:00
...,...,...,...,...,...,...,...,...,...,...,...,...
19757,2024-05-31 14:56:00,3571.4,3540.0,-31.4,2024-05-31,14,56,0,2024,5,202405,14:56:00
19758,2024-05-31 14:57:00,3572.2,3541.2,-31.0,2024-05-31,14,57,0,2024,5,202405,14:57:00
19759,2024-05-31 14:58:00,3570.0,3538.2,-31.8,2024-05-31,14,58,0,2024,5,202405,14:58:00
19760,2024-05-31 14:59:00,3569.2,3537.4,-31.8,2024-05-31,14,59,0,2024,5,202405,14:59:00


In [95]:


# wl(df_month_target['spread'].values, df_month_test.groupby('hour_minte')['spread'].mean().values )

In [86]:
before_target = []
after_target = []
for date in list(df_month_test['date'].unique() ):
    if date<'2024-04-19':
        before_target.append(date)
    else:
        after_target.append(date)

In [87]:
df_month_test_tmp = df_month_test[df_month_test['date'].isin(before_target[:-3])]
df_month_test_tmp

,time,IF00_close,IF01_close,spread,date,hour,minute,Expiration_Date,year,month,year_month,hour_minte
10122,2024-04-01 09:30:00,3555.6,3546.4,-9.2,2024-04-01,9,30,0,2024,4,202404,09:30:00
10123,2024-04-01 09:31:00,3566.4,3559.8,-6.6,2024-04-01,9,31,0,2024,4,202404,09:31:00
10124,2024-04-01 09:32:00,3573.8,3567.2,-6.6,2024-04-01,9,32,0,2024,4,202404,09:32:00
10125,2024-04-01 09:33:00,3576.0,3568.2,-7.8,2024-04-01,9,33,0,2024,4,202404,09:33:00
10126,2024-04-01 09:34:00,3576.0,3569.4,-6.6,2024-04-01,9,34,0,2024,4,202404,09:34:00
...,...,...,...,...,...,...,...,...,...,...,...,...
12286,2024-04-15 14:56:00,3544.2,3539.2,-5.0,2024-04-15,14,56,0,2024,4,202404,14:56:00
12287,2024-04-15 14:57:00,3546.0,3539.0,-7.0,2024-04-15,14,57,0,2024,4,202404,14:57:00
12288,2024-04-15 14:58:00,3545.6,3540.0,-5.6,2024-04-15,14,58,0,2024,4,202404,14:58:00
12289,2024-04-15 14:59:00,3547.2,3541.4,-5.8,2024-04-15,14,59,0,2024,4,202404,14:59:00


In [88]:
wl(df_month_target['spread'].values, df_month_test_tmp.groupby('hour_minte')['spread'].mean().values )
# dangtian  和t-3之前的均值

WilcoxonResult(statistic=10.0, pvalue=3.119988981808843e-41)

In [103]:
df_month_test_tmp2 = df_month_test[df_month_test['date'].isin(before_target[0:-3] )]
wl(df_month_test_tmp2.groupby('hour_minte')['spread'].mean().values,\
    df_month_test[df_month_test['date']==before_target[-1]]['spread'].values)

    # t-1  和t-3之前的均值

WilcoxonResult(statistic=0.0, pvalue=2.7533393932393894e-41)

In [104]:

df_month_test_tmp2 = df_month_test[df_month_test['date'].isin(before_target[0:-3] )]
wl(df_month_test_tmp2.groupby('hour_minte')['spread'].mean().values,\
    df_month_test[df_month_test['date']==before_target[-2]]['spread'].values)

    # t-2  和t-3之前的均值

WilcoxonResult(statistic=180.0, pvalue=2.5724000281269803e-40)

In [110]:
after_target[0],after_target[1],after_target[2]

('2024-04-22', '2024-04-23', '2024-04-24')

In [106]:
df_month_test_tmp2 = df_month_test[df_month_test['date'].isin(before_target[0:-3] )]
wl(df_month_test_tmp2.groupby('hour_minte')['spread'].mean().values,\
    df_month_test[df_month_test['date']==after_target[0]]['spread'].values)
    # t+ 1  与和t-3之前的均值

WilcoxonResult(statistic=0.0, pvalue=2.753713221933144e-41)

In [107]:
df_month_test_tmp2 = df_month_test[df_month_test['date'].isin(before_target[0:-3] )]
wl(df_month_test_tmp2.groupby('hour_minte')['spread'].mean().values,\
    df_month_test[df_month_test['date']==after_target[1]]['spread'].values)
    # t+2 和t-3之前的均值

WilcoxonResult(statistic=0.0, pvalue=2.753793334540499e-41)

In [108]:
df_month_test_tmp2 = df_month_test[df_month_test['date'].isin(before_target[0:-3] )]
wl(df_month_test_tmp2.groupby('hour_minte')['spread'].mean().values,\
    df_month_test[df_month_test['date']==after_target[2]]['spread'].values)

    # t+3 和t-3之前的均值

WilcoxonResult(statistic=0.0, pvalue=2.753553003557335e-41)

## 统计部分时段内的对比

In [144]:
df_month_target

for i in range(1,3):
    for length in [15,30,60]:


        print(before_target[-i], f"前 {length} minutes",wl(df_month_test[df_month_test['date']==before_target[-i]]['spread'].values[0:length],\
            df_month_target['spread'].values[0:length]))



2024-04-18 前 15 minutes WilcoxonResult(statistic=0.0, pvalue=6.103515625e-05)
2024-04-18 前 30 minutes WilcoxonResult(statistic=0.0, pvalue=1.862645149230957e-09)
2024-04-18 前 60 minutes WilcoxonResult(statistic=0.0, pvalue=1.6049222893299423e-11)
2024-04-17 前 15 minutes WilcoxonResult(statistic=0.0, pvalue=6.103515625e-05)
2024-04-17 前 30 minutes WilcoxonResult(statistic=0.0, pvalue=1.862645149230957e-09)
2024-04-17 前 60 minutes WilcoxonResult(statistic=0.0, pvalue=1.5991315303862537e-11)


In [145]:
for i in range(1,3):
    for length in [15,30,60]:


        print(before_target[-i], f"后 {length} minutes",wl(df_month_test[df_month_test['date']==before_target[-i]]['spread'].values[-length:],\
            df_month_target['spread'].values[-length:]))



2024-04-18 后 15 minutes WilcoxonResult(statistic=22.0, pvalue=0.05499177085434167)
2024-04-18 后 30 minutes WilcoxonResult(statistic=141.0, pvalue=0.09794469140952976)
2024-04-18 后 60 minutes WilcoxonResult(statistic=758.5, pvalue=0.5889153536695779)
2024-04-17 后 15 minutes WilcoxonResult(statistic=2.0, pvalue=0.00018310546875)
2024-04-17 后 30 minutes WilcoxonResult(statistic=232.0, pvalue=1.0)
2024-04-17 后 60 minutes WilcoxonResult(statistic=553.0, pvalue=0.007691197252811154)


/Users/gaoxiancheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_morestats.py:4088: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [146]:
df_month_target['minutes'] = df_month_target['hour']*60 + df_month_target['minute']
df_month_test['minutes'] = df_month_test['hour']*60 + df_month_test['minute']


time_ranges = [[13.0,14.0],[10.5,11.5]]

for i in range(1,3):
    for time_range in time_ranges:
        minutes_start = time_range[0]*60
        minutes_end = time_range[1]*60



        print(before_target[-i], f"{minutes_start}-{minutes_end}时间段内",  wl(df_month_test[(df_month_test['date']==before_target[-i])&
        (df_month_test['minutes']>=minutes_start)&(df_month_test['minutes']<minutes_end)]['spread'].values,\
            df_month_target[(df_month_target['minutes']>=minutes_start)&(df_month_target['minutes']<minutes_end)]['spread'].values))

2024-04-18 780.0-840.0时间段内 WilcoxonResult(statistic=809.0, pvalue=0.8893838110309132)
2024-04-18 630.0-690.0时间段内 WilcoxonResult(statistic=47.0, pvalue=3.7947124352518313e-10)
2024-04-17 780.0-840.0时间段内 WilcoxonResult(statistic=58.5, pvalue=1.043135103146265e-09)
2024-04-17 630.0-690.0时间段内 WilcoxonResult(statistic=0.0, pvalue=1.6099733141895078e-11)


/var/folders/71/8c25_pts4j92396n9h1t_88c0000gn/T/ipykernel_1417/2937490392.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_month_target['minutes'] = df_month_target['hour']*60 + df_month_target['minute']


# 对波动率进行做检验

## 读取数据并进行波动率的计算

In [131]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 

df = pd.read_csv('./data/df.csv',sep=',')
df  = df.sort_values(by='time',ascending=True).reset_index(drop=True)

BATCHSIZE = 256
LOOKBACK = 10


date_target = ["2024-02-19",
"2024-03-15",
"2024-04-19",
"2024-05-17",
"2024-06-21",
"2024-07-19",
"2024-08-16",
"2024-09-20",
"2024-10-18",
"2024-11-15",
"2024-12-20",
"2025-01-17"]
df.head()

df['time'] = pd.to_datetime(df['time'])
df['date'] = df['time'].dt.date
df['date'] = df['date'].astype(str)
df['hour'] = df['time'].dt.hour
df['minute'] = df['time'].dt.minute 
# for i in range(1,5):
#     df[f'spread_shift_{i}'] = df.groupby(['hour','minute'])['spread'].shift(i)

df['Expiration_Date'] = df['date'].apply(lambda x: 1 if x in date else 0)
# df.fillna(0,inplace=True)

df['std'] = df['spread'].rolling(window=200).std()
df['volatility'] = df['std'].apply(lambda x: np.log(x))

df.dropna(inplace=True)

df.head()


,time,IF00_close,IF01_close,spread,date,hour,minute,Expiration_Date,std,volatility
199,2024-01-24 14:19:00,3250.6,3245.8,-4.8,2024-01-24,14,19,0,2.284265,0.826044
200,2024-01-24 14:20:00,3245.8,3241.0,-4.8,2024-01-24,14,20,0,2.294348,0.830449
201,2024-01-24 14:21:00,3243.2,3239.0,-4.2,2024-01-24,14,21,0,2.296284,0.831292
202,2024-01-24 14:22:00,3240.8,3235.0,-5.8,2024-01-24,14,22,0,2.300440,0.833100
203,2024-01-24 14:23:00,3243.8,3238.0,-5.8,2024-01-24,14,23,0,2.316842,0.840205


In [133]:
# df['volatility'].plot()

In [134]:
df['year'] = df['time'].dt.year 
df['month'] = df['time'].dt.month 

In [135]:
df['year_month'] = df[['year','month']].apply(lambda x: str(x[0])+ str('0'+ str(x[1] \
    if x[1]<=9 else str(x[1]))),axis=1)

/var/folders/71/8c25_pts4j92396n9h1t_88c0000gn/T/ipykernel_1417/3604794410.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['year_month'] = df[['year','month']].apply(lambda x: str(x[0])+ str('0'+ str(x[1] \
/var/folders/71/8c25_pts4j92396n9h1t_88c0000gn/T/ipykernel_1417/3604794410.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if x[1]<=9 else str(x[1]))),axis=1)


In [136]:
df_month = df[df['year_month']=='202404']

In [137]:
df_month_target = pd.DataFrame()
df_month_test = pd.DataFrame() 
for date in df_month['date'].unique():
    if date in date_target:
        df_month_target = df_month[df_month['date']==date]
    else:
        df_month_test = pd.concat([df_month_test,df_month[df_month['date']==date]],axis=0)

In [138]:
for date in df_month_test['date'].unique():
    test_volatility = df_month_test[df_month_test['date']==date]['volatility'].values 
    target_volatility = df_month_target['volatility'].values 
    if test_volatility.shape==target_volatility.shape:
        print(date, wl(target_volatility,test_volatility))
    else:
        print(date)

2024-04-01 WilcoxonResult(statistic=0.0, pvalue=2.7539802728227173e-41)
2024-04-02 WilcoxonResult(statistic=73.0, pvalue=6.838541429790534e-41)
2024-04-03 WilcoxonResult(statistic=3.0, pvalue=2.8591204628733877e-41)
2024-04-08 WilcoxonResult(statistic=0.0, pvalue=2.7539535665938414e-41)
2024-04-09 WilcoxonResult(statistic=0.0, pvalue=2.7539535665938414e-41)
2024-04-10 WilcoxonResult(statistic=6.0, pvalue=2.9682520081583674e-41)
2024-04-11 WilcoxonResult(statistic=0.0, pvalue=2.7539268606182916e-41)
2024-04-12 WilcoxonResult(statistic=3.0, pvalue=2.8590927484133196e-41)
2024-04-15 WilcoxonResult(statistic=0.0, pvalue=2.7539535665938414e-41)
2024-04-16 WilcoxonResult(statistic=0.0, pvalue=2.7539802728227173e-41)
2024-04-17 WilcoxonResult(statistic=13209.0, pvalue=0.20553608732966466)
2024-04-18 WilcoxonResult(statistic=329.0, pvalue=1.602068716315956e-39)
2024-04-22 WilcoxonResult(statistic=1465.0, pvalue=9.816459871783784e-34)
2024-04-23 WilcoxonResult(statistic=0.0, pvalue=2.7539802728

In [139]:
df_month_test['hour_minte'] = df_month_test['time'].apply(lambda x: str(x)[10:])
df_month_test

,time,IF00_close,IF01_close,spread,date,hour,minute,Expiration_Date,std,volatility,year,month,year_month,hour_minte
10122,2024-04-01 09:30:00,3555.6,3546.4,-9.2,2024-04-01,9,30,0,0.705752,-0.348491,2024,4,202404,09:30:00
10123,2024-04-01 09:31:00,3566.4,3559.8,-6.6,2024-04-01,9,31,0,0.705183,-0.349299,2024,4,202404,09:31:00
10124,2024-04-01 09:32:00,3573.8,3567.2,-6.6,2024-04-01,9,32,0,0.705459,-0.348907,2024,4,202404,09:32:00
10125,2024-04-01 09:33:00,3576.0,3568.2,-7.8,2024-04-01,9,33,0,0.712420,-0.339088,2024,4,202404,09:33:00
10126,2024-04-01 09:34:00,3576.0,3569.4,-6.6,2024-04-01,9,34,0,0.697141,-0.360768,2024,4,202404,09:34:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14937,2024-04-30 14:56:00,3612.0,3601.4,-10.6,2024-04-30,14,56,1,0.668480,-0.402749,2024,4,202404,14:56:00
14938,2024-04-30 14:57:00,3610.6,3600.0,-10.6,2024-04-30,14,57,1,0.669189,-0.401689,2024,4,202404,14:57:00
14939,2024-04-30 14:58:00,3608.4,3598.6,-9.8,2024-04-30,14,58,1,0.664233,-0.409123,2024,4,202404,14:58:00
14940,2024-04-30 14:59:00,3610.8,3599.8,-11.0,2024-04-30,14,59,1,0.665013,-0.407949,2024,4,202404,14:59:00


In [140]:

wl(df_month_target['volatility'].values, df_month_test.groupby('hour_minte')['volatility'].mean().values )

WilcoxonResult(statistic=1.0, pvalue=2.7885925919848393e-41)

In [141]:
before_target = []
after_target = []
for date in list(df_month_test['date'].unique() ):
    if date<'2024-04-19':
        before_target.append(date)
    else:
        after_target.append(date)

In [142]:
df_month_test_tmp = df_month_test[df_month_test['date'].isin(before_target[:-3])]
df_month_test_tmp

,time,IF00_close,IF01_close,spread,date,hour,minute,Expiration_Date,std,volatility,year,month,year_month,hour_minte
10122,2024-04-01 09:30:00,3555.6,3546.4,-9.2,2024-04-01,9,30,0,0.705752,-0.348491,2024,4,202404,09:30:00
10123,2024-04-01 09:31:00,3566.4,3559.8,-6.6,2024-04-01,9,31,0,0.705183,-0.349299,2024,4,202404,09:31:00
10124,2024-04-01 09:32:00,3573.8,3567.2,-6.6,2024-04-01,9,32,0,0.705459,-0.348907,2024,4,202404,09:32:00
10125,2024-04-01 09:33:00,3576.0,3568.2,-7.8,2024-04-01,9,33,0,0.712420,-0.339088,2024,4,202404,09:33:00
10126,2024-04-01 09:34:00,3576.0,3569.4,-6.6,2024-04-01,9,34,0,0.697141,-0.360768,2024,4,202404,09:34:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12286,2024-04-15 14:56:00,3544.2,3539.2,-5.0,2024-04-15,14,56,0,0.679095,-0.386994,2024,4,202404,14:56:00
12287,2024-04-15 14:57:00,3546.0,3539.0,-7.0,2024-04-15,14,57,0,0.684589,-0.378936,2024,4,202404,14:57:00
12288,2024-04-15 14:58:00,3545.6,3540.0,-5.6,2024-04-15,14,58,0,0.684235,-0.379455,2024,4,202404,14:58:00
12289,2024-04-15 14:59:00,3547.2,3541.4,-5.8,2024-04-15,14,59,0,0.684235,-0.379455,2024,4,202404,14:59:00


In [143]:
wl(df_month_target['volatility'].values, df_month_test_tmp.groupby('hour_minte')['volatility'].mean().values )
# dangtian  和t-3之前的均值

WilcoxonResult(statistic=0.0, pvalue=2.7539802728227173e-41)

In [147]:
df_month_test_tmp2 = df_month_test[df_month_test['date'].isin(before_target[0:-3] )]
wl(df_month_test_tmp2.groupby('hour_minte')['volatility'].mean().values,\
    df_month_test[df_month_test['date']==before_target[-1]]['volatility'].values)

    # t-1  和t-3之前的均值

WilcoxonResult(statistic=3862.0, pvalue=4.4422919141296776e-23)

In [148]:
df_month_test_tmp2 = df_month_test[df_month_test['date'].isin(before_target[0:-3] )]
wl(df_month_test_tmp2.groupby('hour_minte')['volatility'].mean().values,\
    df_month_test[df_month_test['date']==before_target[-2]]['volatility'].values)

    # t-2  和t-3之前的均值

WilcoxonResult(statistic=370.0, pvalue=2.6412507393318968e-39)

In [149]:
df_month_test_tmp2 = df_month_test[df_month_test['date'].isin(before_target[0:-3] )]
wl(df_month_test_tmp2.groupby('hour_minte')['volatility'].mean().values,\
    df_month_test[df_month_test['date']==after_target[0]]['volatility'].values)
    # t+ 1  与和t-3之前的均值

WilcoxonResult(statistic=441.0, pvalue=6.256763488529448e-39)

In [150]:
df_month_test_tmp2 = df_month_test[df_month_test['date'].isin(before_target[0:-3] )]
wl(df_month_test_tmp2.groupby('hour_minte')['volatility'].mean().values,\
    df_month_test[df_month_test['date']==after_target[1]]['volatility'].values)
    # t+2 和t-3之前的均值

WilcoxonResult(statistic=2579.0, pvalue=1.6087251090096632e-28)

In [151]:
df_month_target

for i in range(1,3):
    for length in [15,30,60]:


        print(before_target[-i], f"前 {length} minutes",wl(df_month_test[df_month_test['date']==before_target[-i]]['volatility'].values[0:length],\
            df_month_target['volatility'].values[0:length]))



2024-04-18 前 15 minutes WilcoxonResult(statistic=4.0, pvalue=0.00042724609375)
2024-04-18 前 30 minutes WilcoxonResult(statistic=200.0, pvalue=0.5158484429121017)
2024-04-18 前 60 minutes WilcoxonResult(statistic=232.0, pvalue=4.956868670525348e-07)
2024-04-17 前 15 minutes WilcoxonResult(statistic=0.0, pvalue=6.103515625e-05)
2024-04-17 前 30 minutes WilcoxonResult(statistic=0.0, pvalue=1.862645149230957e-09)
2024-04-17 前 60 minutes WilcoxonResult(statistic=449.0, pvalue=0.0006024544311607902)


In [152]:
for i in range(1,3):
    for length in [15,30,60]:


        print(before_target[-i], f"后 {length} minutes",wl(df_month_test[df_month_test['date']==before_target[-i]]['volatility'].values[-length:],\
            df_month_target['volatility'].values[-length:]))


2024-04-18 后 15 minutes WilcoxonResult(statistic=0.0, pvalue=6.103515625e-05)
2024-04-18 后 30 minutes WilcoxonResult(statistic=0.0, pvalue=1.862645149230957e-09)
2024-04-18 后 60 minutes WilcoxonResult(statistic=0.0, pvalue=1.6295557943119345e-11)
2024-04-17 后 15 minutes WilcoxonResult(statistic=0.0, pvalue=6.103515625e-05)
2024-04-17 后 30 minutes WilcoxonResult(statistic=0.0, pvalue=1.862645149230957e-09)
2024-04-17 后 60 minutes WilcoxonResult(statistic=0.0, pvalue=1.6295557943119345e-11)


In [153]:
df_month_target['minutes'] = df_month_target['hour']*60 + df_month_target['minute']
df_month_test['minutes'] = df_month_test['hour']*60 + df_month_test['minute']


time_ranges = [[13.0,14.0],[10.5,11.5]]

for i in range(1,3):
    for time_range in time_ranges:
        minutes_start = time_range[0]*60
        minutes_end = time_range[1]*60



        print(before_target[-i], f"{minutes_start}-{minutes_end}时间段内",  wl(df_month_test[(df_month_test['date']==before_target[-i])&
        (df_month_test['minutes']>=minutes_start)&(df_month_test['minutes']<minutes_end)]['volatility'].values,\
            df_month_target[(df_month_target['minutes']>=minutes_start)&(df_month_target['minutes']<minutes_end)]['volatility'].values))

2024-04-18 780.0-840.0时间段内 WilcoxonResult(statistic=0.0, pvalue=2.3900953516978203e-11)
2024-04-18 630.0-690.0时间段内 WilcoxonResult(statistic=0.0, pvalue=1.6295557943119345e-11)
2024-04-17 780.0-840.0时间段内 WilcoxonResult(statistic=370.0, pvalue=0.00010140401371014285)
2024-04-17 630.0-690.0时间段内 WilcoxonResult(statistic=0.0, pvalue=1.6295557943119345e-11)


/var/folders/71/8c25_pts4j92396n9h1t_88c0000gn/T/ipykernel_1417/1928755197.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_month_target['minutes'] = df_month_target['hour']*60 + df_month_target['minute']
